# POG Model Analysis Notebook

Analysis of Point-of-Gaze (POG) model predictions from calibration sessions.

**Models evaluated:**
- `Best_POG_model_scripted.pt` (ITrackerMultiHeadAttention)

**Data:**
- Two calibration sessions with POG predictions in centimeters
- Includes pupillometry (PIR), gaze tracking, and ground truth calibration dots

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11

# Load data
DATA_DIR = Path('/media/a/saw/pog')

files = [
    '0014c300-76ca-456e-9ada-89db12_calibration_with_pog_cm.parquet',
    '0132028e-95b1-4f9e-bdd6-1ddc9a_calibration_with_pog_cm.parquet'
]

dfs = []
for f in files:
    df = pd.read_parquet(DATA_DIR / f)
    df['source_file'] = f.split('_')[0]
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)
print(f"Total samples: {len(df):,}")
print(f"Columns: {len(df.columns)}")
print(f"Sessions: {df['source_file'].nunique()}")

## 1. Data Overview

In [ ]:
# Key columns for analysis
key_cols = [
    'pog_x_coord', 'pog_y_coord',           # POG in pixels
    'pog_x_cm', 'pog_y_cm',                 # POG in centimeters
    'displayed_cal_dot_at_x', 'displayed_cal_dot_at_y',  # Ground truth dots
    'device_type', 'pog_model', 'seg_model',
    'pir_left', 'pir_right',                # Pupil-to-iris ratio
    'pupil_area_left', 'pupil_area_right',
    'distance', 'lumens'
]

print("Key columns summary:")
df[key_cols].describe().round(3)

In [ ]:
# Device and model info
print("Devices:")
print(df['device_type'].value_counts())
print("\nPOG Models:")
print(df['pog_model'].value_counts())
print("\nSegmentation Models:")
print(df['seg_model'].value_counts())

## 2. POG Prediction Distribution

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# POG in pixels
ax = axes[0, 0]
ax.scatter(df['pog_x_coord'], df['pog_y_coord'], alpha=0.3, s=5, c='blue')
ax.set_xlabel('POG X (pixels)')
ax.set_ylabel('POG Y (pixels)')
ax.set_title('POG Predictions (pixels)')
ax.invert_yaxis()

# POG in cm
ax = axes[0, 1]
ax.scatter(df['pog_x_cm'], df['pog_y_cm'], alpha=0.3, s=5, c='green')
ax.set_xlabel('POG X (cm)')
ax.set_ylabel('POG Y (cm)')
ax.set_title('POG Predictions (centimeters)')

# Calibration dots (ground truth)
ax = axes[1, 0]
cal_dots = df[['displayed_cal_dot_at_x', 'displayed_cal_dot_at_y']].dropna()
ax.scatter(cal_dots['displayed_cal_dot_at_x'], cal_dots['displayed_cal_dot_at_y'], 
           alpha=0.5, s=10, c='red')
ax.set_xlabel('Calibration Dot X (pixels)')
ax.set_ylabel('Calibration Dot Y (pixels)')
ax.set_title('Calibration Dot Positions (Ground Truth)')
ax.invert_yaxis()

# Distribution histograms
ax = axes[1, 1]
ax.hist(df['pog_x_cm'].dropna(), bins=50, alpha=0.5, label='X (cm)', density=True)
ax.hist(df['pog_y_cm'].dropna(), bins=50, alpha=0.5, label='Y (cm)', density=True)
ax.set_xlabel('POG Coordinate (cm)')
ax.set_ylabel('Density')
ax.set_title('POG Coordinate Distributions')
ax.legend()

plt.tight_layout()
plt.savefig('pog_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

## 3. POG Error Analysis (vs Calibration Dots)

In [ ]:
# Filter to calibration events with ground truth
cal_df = df[df['displayed_cal_dot_at_x'].notna() & df['pog_x_coord'].notna()].copy()
print(f"Calibration samples with ground truth: {len(cal_df):,}")

# Calculate error in pixels
cal_df['error_x_px'] = cal_df['pog_x_coord'] - cal_df['displayed_cal_dot_at_x']
cal_df['error_y_px'] = cal_df['pog_y_coord'] - cal_df['displayed_cal_dot_at_y']
cal_df['error_euclidean_px'] = np.sqrt(cal_df['error_x_px']**2 + cal_df['error_y_px']**2)

print("\nError Statistics (pixels):")
print(f"  Mean X error: {cal_df['error_x_px'].mean():.2f} px")
print(f"  Mean Y error: {cal_df['error_y_px'].mean():.2f} px")
print(f"  Mean Euclidean error: {cal_df['error_euclidean_px'].mean():.2f} px")
print(f"  Median Euclidean error: {cal_df['error_euclidean_px'].median():.2f} px")
print(f"  Std Euclidean error: {cal_df['error_euclidean_px'].std():.2f} px")

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Error distribution
ax = axes[0]
ax.hist(cal_df['error_euclidean_px'], bins=50, edgecolor='black', alpha=0.7)
ax.axvline(cal_df['error_euclidean_px'].mean(), color='red', linestyle='--', 
           label=f"Mean: {cal_df['error_euclidean_px'].mean():.1f} px")
ax.axvline(cal_df['error_euclidean_px'].median(), color='orange', linestyle='--',
           label=f"Median: {cal_df['error_euclidean_px'].median():.1f} px")
ax.set_xlabel('Euclidean Error (pixels)')
ax.set_ylabel('Count')
ax.set_title('POG Error Distribution')
ax.legend()

# Error scatter
ax = axes[1]
ax.scatter(cal_df['error_x_px'], cal_df['error_y_px'], alpha=0.3, s=5)
ax.axhline(0, color='red', linestyle='-', alpha=0.5)
ax.axvline(0, color='red', linestyle='-', alpha=0.5)
ax.set_xlabel('X Error (pixels)')
ax.set_ylabel('Y Error (pixels)')
ax.set_title('Error Vector Distribution')
ax.set_aspect('equal')

# Error by calibration dot position
ax = axes[2]
scatter = ax.scatter(cal_df['displayed_cal_dot_at_x'], cal_df['displayed_cal_dot_at_y'],
                     c=cal_df['error_euclidean_px'], cmap='RdYlGn_r', alpha=0.5, s=10)
plt.colorbar(scatter, ax=ax, label='Error (px)')
ax.set_xlabel('Calibration Dot X')
ax.set_ylabel('Calibration Dot Y')
ax.set_title('Error by Screen Position')
ax.invert_yaxis()

plt.tight_layout()
plt.savefig('pog_error_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

## 4. Error by Calibration Dot

In [ ]:
# Group by unique calibration dot positions
cal_df['dot_pos'] = cal_df['displayed_cal_dot_at_x'].astype(str) + ',' + cal_df['displayed_cal_dot_at_y'].astype(str)

dot_errors = cal_df.groupby('dot_pos').agg({
    'error_euclidean_px': ['mean', 'std', 'count'],
    'displayed_cal_dot_at_x': 'first',
    'displayed_cal_dot_at_y': 'first'
}).round(2)

dot_errors.columns = ['mean_error', 'std_error', 'count', 'dot_x', 'dot_y']
dot_errors = dot_errors.sort_values('mean_error')

print("Error by Calibration Dot Position:")
print(dot_errors.to_string())

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

scatter = ax.scatter(dot_errors['dot_x'], dot_errors['dot_y'], 
                     c=dot_errors['mean_error'], s=dot_errors['count']/5,
                     cmap='RdYlGn_r', alpha=0.8, edgecolors='black')

# Add error labels
for idx, row in dot_errors.iterrows():
    ax.annotate(f"{row['mean_error']:.0f}", 
                (row['dot_x'], row['dot_y']),
                textcoords="offset points", xytext=(0,10), ha='center', fontsize=9)

plt.colorbar(scatter, label='Mean Error (px)')
ax.set_xlabel('Calibration Dot X (pixels)')
ax.set_ylabel('Calibration Dot Y (pixels)')
ax.set_title('Mean POG Error by Calibration Dot Position\n(size = sample count)')
ax.invert_yaxis()

plt.tight_layout()
plt.savefig('pog_error_by_dot.png', dpi=150, bbox_inches='tight')
plt.show()

## 5. Pupillometry Analysis (PIR)

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# PIR distributions
ax = axes[0, 0]
ax.hist(df['pir_left'].dropna(), bins=50, alpha=0.5, label='Left Eye', density=True)
ax.hist(df['pir_right'].dropna(), bins=50, alpha=0.5, label='Right Eye', density=True)
ax.set_xlabel('Pupil-to-Iris Ratio (PIR)')
ax.set_ylabel('Density')
ax.set_title('PIR Distribution')
ax.legend()

# PIR over time (frame index)
ax = axes[0, 1]
sample_session = df[df['source_file'] == df['source_file'].iloc[0]].copy()
ax.plot(sample_session['frame_index'], sample_session['pir_left_savgol'], 
        alpha=0.7, label='Left (smoothed)', linewidth=0.8)
ax.plot(sample_session['frame_index'], sample_session['pir_right_savgol'], 
        alpha=0.7, label='Right (smoothed)', linewidth=0.8)
ax.set_xlabel('Frame Index')
ax.set_ylabel('PIR')
ax.set_title('PIR Over Time (Sample Session)')
ax.legend()

# Left vs Right PIR correlation
ax = axes[1, 0]
pir_valid = df[['pir_left', 'pir_right']].dropna()
ax.scatter(pir_valid['pir_left'], pir_valid['pir_right'], alpha=0.3, s=5)
ax.plot([0, 0.6], [0, 0.6], 'r--', label='y=x')
ax.set_xlabel('PIR Left')
ax.set_ylabel('PIR Right')
ax.set_title(f"Left vs Right PIR (r={pir_valid['pir_left'].corr(pir_valid['pir_right']):.3f})")
ax.legend()

# Pupil area over time
ax = axes[1, 1]
ax.plot(sample_session['frame_index'], sample_session['pupil_area_left'], 
        alpha=0.7, label='Left', linewidth=0.8)
ax.plot(sample_session['frame_index'], sample_session['pupil_area_right'], 
        alpha=0.7, label='Right', linewidth=0.8)
ax.set_xlabel('Frame Index')
ax.set_ylabel('Pupil Area (pixels²)')
ax.set_title('Pupil Area Over Time')
ax.legend()

plt.tight_layout()
plt.savefig('pupillometry_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. Session Comparison

In [ ]:
# Compare sessions
session_stats = df.groupby('source_file').agg({
    'pog_x_cm': ['mean', 'std'],
    'pog_y_cm': ['mean', 'std'],
    'pir_left': 'mean',
    'pir_right': 'mean',
    'distance': 'mean',
    'lumens': 'mean',
    'frame_index': 'count'
}).round(3)

session_stats.columns = ['pog_x_mean', 'pog_x_std', 'pog_y_mean', 'pog_y_std',
                         'pir_left_mean', 'pir_right_mean', 'distance_mean', 
                         'lumens_mean', 'frame_count']

print("Session Statistics:")
print(session_stats.to_string())

In [ ]:
# Calculate error stats per session
cal_session_stats = cal_df.groupby('source_file').agg({
    'error_euclidean_px': ['mean', 'median', 'std'],
    'error_x_px': 'mean',
    'error_y_px': 'mean'
}).round(2)

cal_session_stats.columns = ['error_mean', 'error_median', 'error_std', 'bias_x', 'bias_y']

print("\nCalibration Error by Session:")
print(cal_session_stats.to_string())

## 7. Summary Statistics

In [ ]:
print("="*60)
print("POG MODEL EVALUATION SUMMARY")
print("="*60)
print(f"\nModel: {df['pog_model'].iloc[0]}")
print(f"Segmentation Model: {df['seg_model'].iloc[0]}")
print(f"Detector: {df['detector_model'].iloc[0]}")
print(f"\nTotal frames analyzed: {len(df):,}")
print(f"Sessions: {df['source_file'].nunique()}")
print(f"Devices: {df['device_type'].unique().tolist()}")

print(f"\n--- POG Output Statistics ---")
print(f"POG X (cm): mean={df['pog_x_cm'].mean():.3f}, std={df['pog_x_cm'].std():.3f}")
print(f"POG Y (cm): mean={df['pog_y_cm'].mean():.3f}, std={df['pog_y_cm'].std():.3f}")

print(f"\n--- Calibration Error (vs Ground Truth Dots) ---")
print(f"Mean Euclidean Error: {cal_df['error_euclidean_px'].mean():.1f} pixels")
print(f"Median Euclidean Error: {cal_df['error_euclidean_px'].median():.1f} pixels")
print(f"Std Euclidean Error: {cal_df['error_euclidean_px'].std():.1f} pixels")
print(f"X Bias: {cal_df['error_x_px'].mean():.1f} pixels")
print(f"Y Bias: {cal_df['error_y_px'].mean():.1f} pixels")

print(f"\n--- Pupillometry ---")
print(f"PIR Left: mean={df['pir_left'].mean():.3f}, std={df['pir_left'].std():.3f}")
print(f"PIR Right: mean={df['pir_right'].mean():.3f}, std={df['pir_right'].std():.3f}")
print(f"L-R Correlation: {df['pir_left'].corr(df['pir_right']):.3f}")

print(f"\n--- Session Conditions ---")
print(f"Distance: mean={df['distance'].mean():.2f} cm")
print(f"Lumens: mean={df['lumens'].mean():.1f}")

In [ ]:
# Save summary to CSV
summary_data = {
    'metric': [
        'Total Frames', 'Sessions', 'Model',
        'Mean Error (px)', 'Median Error (px)', 'Std Error (px)',
        'X Bias (px)', 'Y Bias (px)',
        'POG X Mean (cm)', 'POG Y Mean (cm)',
        'PIR Left Mean', 'PIR Right Mean'
    ],
    'value': [
        len(df), df['source_file'].nunique(), df['pog_model'].iloc[0],
        round(cal_df['error_euclidean_px'].mean(), 1),
        round(cal_df['error_euclidean_px'].median(), 1),
        round(cal_df['error_euclidean_px'].std(), 1),
        round(cal_df['error_x_px'].mean(), 1),
        round(cal_df['error_y_px'].mean(), 1),
        round(df['pog_x_cm'].mean(), 3),
        round(df['pog_y_cm'].mean(), 3),
        round(df['pir_left'].mean(), 3),
        round(df['pir_right'].mean(), 3)
    ]
}

summary_df = pd.DataFrame(summary_data)
summary_df.to_csv('pog_analysis_summary.csv', index=False)
print("Summary saved to pog_analysis_summary.csv")
print(summary_df.to_string(index=False))